In [183]:
#!/usr/bin/python3 -B

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

In [184]:
# Data wrangling brought to you by the1owl
# https://www.kaggle.com/the1owl/surprise-me

data = {
    'tra':
    pd.read_csv('../input/air_visit_data.csv'),
    'as':
    pd.read_csv('../input/air_store_info.csv'),
    'hs':
    pd.read_csv('../input/hpg_store_info.csv'),
    'ar':
    pd.read_csv('../input/air_reserve.csv'),
    'hr':
    pd.read_csv('../input/hpg_reserve.csv'),
    'id':
    pd.read_csv('../input/store_id_relation.csv'),
    'tes':
    pd.read_csv('../input/sample_submission.csv'),
    'hol':
    pd.read_csv('../input/date_info.csv').rename(columns={
        'calendar_date': 'visit_date'
    })
}

In [185]:
data['hr'].head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [186]:
data['id'].head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


In [187]:
data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

In [188]:
data['hr'].head(10)

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926
5,hpg_878cc70b1abc76f7,2016-01-12 19:00:00,2016-01-04 10:00:00,3,air_db80363d35f10926
6,hpg_878cc70b1abc76f7,2016-01-15 18:00:00,2016-01-07 14:00:00,9,air_db80363d35f10926
7,hpg_878cc70b1abc76f7,2016-01-16 20:00:00,2016-01-15 07:00:00,3,air_db80363d35f10926
8,hpg_878cc70b1abc76f7,2016-01-18 18:00:00,2016-01-13 11:00:00,2,air_db80363d35f10926
9,hpg_878cc70b1abc76f7,2016-01-22 18:00:00,2016-01-14 20:00:00,3,air_db80363d35f10926


In [189]:
data['ar'].head(10)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5
5,air_db80363d35f10926,2016-01-02 01:00:00,2016-01-01 16:00:00,2
6,air_db80363d35f10926,2016-01-02 01:00:00,2016-01-01 15:00:00,4
7,air_3bb99a1fe0583897,2016-01-02 16:00:00,2016-01-02 14:00:00,2
8,air_3bb99a1fe0583897,2016-01-02 16:00:00,2016-01-01 20:00:00,2
9,air_2b8b29ddfd35018e,2016-01-02 17:00:00,2016-01-02 17:00:00,2


In [190]:
for df in ['ar', 'hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(
        lambda r: (r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
    data[df] = data[df].groupby(
        ['air_store_id', 'visit_datetime'], as_index=False)[[
            'reserve_datetime_diff', 'reserve_visitors'
        ]].sum().rename(columns={
            'visit_datetime': 'visit_date'
        })
    print(data[df].head())

           air_store_id  visit_date  reserve_datetime_diff  reserve_visitors
0  air_00a91d42b08b08d9  2016-10-31                      0                 2
1  air_00a91d42b08b08d9  2016-12-05                      4                 9
2  air_00a91d42b08b08d9  2016-12-14                      6                18
3  air_00a91d42b08b08d9  2016-12-17                      6                 2
4  air_00a91d42b08b08d9  2016-12-20                      2                 4
           air_store_id  visit_date  reserve_datetime_diff  reserve_visitors
0  air_00a91d42b08b08d9  2016-01-14                      3                 2
1  air_00a91d42b08b08d9  2016-01-15                      6                 4
2  air_00a91d42b08b08d9  2016-01-16                      3                 2
3  air_00a91d42b08b08d9  2016-01-22                      3                 2
4  air_00a91d42b08b08d9  2016-01-29                      6                 5


In [191]:
data['tra'].head(10)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
5,air_ba937bf13d40fb24,2016-01-19,9
6,air_ba937bf13d40fb24,2016-01-20,31
7,air_ba937bf13d40fb24,2016-01-21,21
8,air_ba937bf13d40fb24,2016-01-22,18
9,air_ba937bf13d40fb24,2016-01-23,26


In [192]:
data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])

In [193]:
data['tra'].head(10)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
5,air_ba937bf13d40fb24,2016-01-19,9
6,air_ba937bf13d40fb24,2016-01-20,31
7,air_ba937bf13d40fb24,2016-01-21,21
8,air_ba937bf13d40fb24,2016-01-22,18
9,air_ba937bf13d40fb24,2016-01-23,26


In [194]:
data['tra']['day'] = data['tra']['visit_date'].dt.day
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

In [195]:
data['tra'].head(10)

,air_store_id,visit_date,visitors,day,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,13,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,14,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,15,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,16,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,18,0,2016,1
5,air_ba937bf13d40fb24,2016-01-19,9,19,1,2016,1
6,air_ba937bf13d40fb24,2016-01-20,31,20,2,2016,1
7,air_ba937bf13d40fb24,2016-01-21,21,21,3,2016,1
8,air_ba937bf13d40fb24,2016-01-22,18,22,4,2016,1
9,air_ba937bf13d40fb24,2016-01-23,26,23,5,2016,1


In [196]:
data['tes'].head(10)

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0
5,air_00a91d42b08b08d9_2017-04-28,0
6,air_00a91d42b08b08d9_2017-04-29,0
7,air_00a91d42b08b08d9_2017-04-30,0
8,air_00a91d42b08b08d9_2017-05-01,0
9,air_00a91d42b08b08d9_2017-05-02,0


In [197]:
data['tes']['visit_date'] = data['tes']['id'].map(
    lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(
    lambda x: '_'.join(x.split('_')[:2]))

In [198]:
data['tes'].head(10)

,id,visitors,visit_date,air_store_id
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9
5,air_00a91d42b08b08d9_2017-04-28,0,2017-04-28,air_00a91d42b08b08d9
6,air_00a91d42b08b08d9_2017-04-29,0,2017-04-29,air_00a91d42b08b08d9
7,air_00a91d42b08b08d9_2017-04-30,0,2017-04-30,air_00a91d42b08b08d9
8,air_00a91d42b08b08d9_2017-05-01,0,2017-05-01,air_00a91d42b08b08d9
9,air_00a91d42b08b08d9_2017-05-02,0,2017-05-02,air_00a91d42b08b08d9


In [199]:
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['day'] = data['tes']['visit_date'].dt.day
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

In [200]:
data['tes'].head(10)

,id,visitors,visit_date,air_store_id,day,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,23,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,24,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,25,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,26,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,27,3,2017,4
5,air_00a91d42b08b08d9_2017-04-28,0,2017-04-28,air_00a91d42b08b08d9,28,4,2017,4
6,air_00a91d42b08b08d9_2017-04-29,0,2017-04-29,air_00a91d42b08b08d9,29,5,2017,4
7,air_00a91d42b08b08d9_2017-04-30,0,2017-04-30,air_00a91d42b08b08d9,30,6,2017,4
8,air_00a91d42b08b08d9_2017-05-01,0,2017-05-01,air_00a91d42b08b08d9,1,0,2017,5
9,air_00a91d42b08b08d9_2017-05-02,0,2017-05-02,air_00a91d42b08b08d9,2,1,2017,5


In [201]:
unique_stores = data['tes']['air_store_id'].unique()

In [202]:
len(unique_stores)

821

In [203]:
len(data['tra']['air_store_id'].unique())

829

In [204]:
stores = pd.concat(
    [
        pd.DataFrame({
            'air_store_id': unique_stores,
            'dow': [i] * len(unique_stores)
        }) for i in range(7)
    ],
    axis=0,
    ignore_index=True).reset_index(drop=True)

In [205]:
len(stores)

5747

In [206]:
stores.head(10)

,air_store_id,dow
0,air_00a91d42b08b08d9,0
1,air_0164b9927d20bcc3,0
2,air_0241aa3964b7f861,0
3,air_0328696196e46f18,0
4,air_034a3d5b40d5b1b1,0
5,air_036d4f1ee7285390,0
6,air_0382c794b73b51ad,0
7,air_03963426c9312048,0
8,air_04341b588bde96cd,0
9,air_049f6d5b402a31b2,0


In [207]:
#sure it can be compressed...
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].min().rename(columns={
        'visitors': 'min_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

In [208]:
stores.head(10)

,air_store_id,dow,min_visitors
0,air_00a91d42b08b08d9,0,1.0
1,air_0164b9927d20bcc3,0,2.0
2,air_0241aa3964b7f861,0,2.0
3,air_0328696196e46f18,0,2.0
4,air_034a3d5b40d5b1b1,0,1.0
5,air_036d4f1ee7285390,0,4.0
6,air_0382c794b73b51ad,0,1.0
7,air_03963426c9312048,0,2.0
8,air_04341b588bde96cd,0,5.0
9,air_049f6d5b402a31b2,0,2.0


In [209]:
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].mean().rename(columns={
        'visitors': 'mean_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

In [210]:
stores.head(10)

,air_store_id,dow,min_visitors,mean_visitors
0,air_00a91d42b08b08d9,0,1.0,22.457143
1,air_0164b9927d20bcc3,0,2.0,7.500000
2,air_0241aa3964b7f861,0,2.0,8.920635
3,air_0328696196e46f18,0,2.0,6.416667
4,air_034a3d5b40d5b1b1,0,1.0,11.864865
5,air_036d4f1ee7285390,0,4.0,19.600000
6,air_0382c794b73b51ad,0,1.0,20.795455
7,air_03963426c9312048,0,2.0,26.030303
8,air_04341b588bde96cd,0,5.0,35.417910
9,air_049f6d5b402a31b2,0,2.0,9.027778


In [211]:
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].median().rename(columns={
        'visitors': 'median_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].max().rename(columns={
        'visitors': 'max_visitors'
    })
stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

In [212]:
stores.head(10)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0
5,air_036d4f1ee7285390,0,4.0,19.600000,19.0,38.0
6,air_0382c794b73b51ad,0,1.0,20.795455,21.0,47.0
7,air_03963426c9312048,0,2.0,26.030303,26.0,70.0
8,air_04341b588bde96cd,0,5.0,35.417910,33.0,76.0
9,air_049f6d5b402a31b2,0,2.0,9.027778,9.0,20.0


In [213]:
tmp = data['tra'].groupby(
    ['air_store_id', 'dow'],
    as_index=False)['visitors'].count().rename(columns={
        'visitors': 'count_observations'
    })

tmp['mode_visitors'] = pd.DataFrame(
    data['tra'].groupby(['air_store_id', 'dow'],
    as_index=False)['visitors'].apply(lambda r: mode(r)[0][0]).values, columns=['mode_visitors'])['mode_visitors']

stores = pd.merge(stores, tmp, how='left', on=['air_store_id', 'dow'])

In [214]:
stores.head(10)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,mode_visitors
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,18.0
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,2.0
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0,6.0
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0,2.0
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,6.0
5,air_036d4f1ee7285390,0,4.0,19.600000,19.0,38.0,40.0,19.0
6,air_0382c794b73b51ad,0,1.0,20.795455,21.0,47.0,44.0,21.0
7,air_03963426c9312048,0,2.0,26.030303,26.0,70.0,66.0,11.0
8,air_04341b588bde96cd,0,5.0,35.417910,33.0,76.0,67.0,30.0
9,air_049f6d5b402a31b2,0,2.0,9.027778,9.0,20.0,36.0,2.0


In [215]:
stores = pd.merge(stores, data['as'], how='left', on=['air_store_id'])

In [216]:
data['as'].head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [217]:
stores.head(10)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,18.0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,2.0,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0,6.0,Izakaya,Tōkyō-to Taitō-ku Higashiueno,35.712607,139.779996
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0,2.0,Dining bar,Ōsaka-fu Ōsaka-shi Nakanochō,34.701279,135.528090
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,6.0,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Ōhiraki,34.692337,135.472229
5,air_036d4f1ee7285390,0,4.0,19.600000,19.0,38.0,40.0,19.0,Cafe/Sweets,Hyōgo-ken Takarazuka-shi Tōyōchō,34.799767,135.360073
6,air_0382c794b73b51ad,0,1.0,20.795455,21.0,47.0,44.0,21.0,Cafe/Sweets,Tōkyō-to Setagaya-ku Okusawa,35.602125,139.671958
7,air_03963426c9312048,0,2.0,26.030303,26.0,70.0,66.0,11.0,Izakaya,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386245,132.455018
8,air_04341b588bde96cd,0,5.0,35.417910,33.0,76.0,67.0,30.0,Izakaya,Tōkyō-to Nerima-ku Toyotamakita,35.735623,139.651658
9,air_049f6d5b402a31b2,0,2.0,9.027778,9.0,20.0,36.0,2.0,Japanese food,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813


In [218]:
lbl = preprocessing.LabelEncoder()
stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])

In [219]:
stores.head(10)

,air_store_id,dow,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude
0,air_00a91d42b08b08d9,0,1.0,22.457143,19.0,47.0,35.0,18.0,6,44,35.694003,139.753595
1,air_0164b9927d20bcc3,0,2.0,7.500000,6.0,19.0,20.0,2.0,6,62,35.658068,139.751599
2,air_0241aa3964b7f861,0,2.0,8.920635,8.0,23.0,63.0,6.0,7,82,35.712607,139.779996
3,air_0328696196e46f18,0,2.0,6.416667,4.0,27.0,12.0,2.0,4,98,34.701279,135.528090
4,air_034a3d5b40d5b1b1,0,1.0,11.864865,10.0,66.0,37.0,6.0,2,102,34.692337,135.472229
5,air_036d4f1ee7285390,0,4.0,19.600000,19.0,38.0,40.0,19.0,2,31,34.799767,135.360073
6,air_0382c794b73b51ad,0,1.0,20.795455,21.0,47.0,44.0,21.0,2,68,35.602125,139.671958
7,air_03963426c9312048,0,2.0,26.030303,26.0,70.0,66.0,11.0,7,15,34.386245,132.455018
8,air_04341b588bde96cd,0,5.0,35.417910,33.0,76.0,67.0,30.0,7,66,35.735623,139.651658
9,air_049f6d5b402a31b2,0,2.0,9.027778,9.0,20.0,36.0,2.0,8,0,33.589216,130.392813


In [220]:
len(stores)

5747

In [221]:
data['hol'].head()

,visit_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [222]:
data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date

In [223]:
data['hol']['day_of_week'] = data['hol']['day_of_week'] + 1

In [224]:
#data['hol']['holiday_flg'] = data['hol']['holiday_flg'] + 1

In [225]:
data['hol'].head(10)

,visit_date,day_of_week,holiday_flg
0,2016-01-01,1,1
1,2016-01-02,3,1
2,2016-01-03,4,1
3,2016-01-04,2,0
4,2016-01-05,6,0
5,2016-01-06,7,0
6,2016-01-07,5,0
7,2016-01-08,1,0
8,2016-01-09,3,0
9,2016-01-10,4,0


In [226]:
len(data['hol']['visit_date'].unique())

517

In [227]:
data['tra'].head(10)

,air_store_id,visit_date,visitors,day,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,13,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,14,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,15,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,16,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,18,0,2016,1
5,air_ba937bf13d40fb24,2016-01-19,9,19,1,2016,1
6,air_ba937bf13d40fb24,2016-01-20,31,20,2,2016,1
7,air_ba937bf13d40fb24,2016-01-21,21,21,3,2016,1
8,air_ba937bf13d40fb24,2016-01-22,18,22,4,2016,1
9,air_ba937bf13d40fb24,2016-01-23,26,23,5,2016,1


In [228]:
len(data['tra']['visit_date'].unique())

478

In [229]:
data['tes'].head()

,id,visitors,visit_date,air_store_id,day,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,23,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,24,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,25,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,26,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,27,3,2017,4


In [230]:
len(data['tes']['visit_date'].unique())

39

In [231]:
train = pd.merge(data['tra'], data['hol'], how='left', on=['visit_date'])
test = pd.merge(data['tes'], data['hol'], how='left', on=['visit_date'])

train = pd.merge(train, stores, how='left', on=['air_store_id', 'dow'])
test = pd.merge(test, stores, how='left', on=['air_store_id', 'dow'])

for df in ['ar', 'hr']:
    train = pd.merge(
        train, data[df], how='left', on=['air_store_id', 'visit_date'])
    test = pd.merge(
        test, data[df], how='left', on=['air_store_id', 'visit_date'])

col = [
    c for c in train
    if c not in ['id', 'air_store_id', 'visit_date', 'visitors']
]
train = train.fillna(-1)
test = test.fillna(-1)

In [232]:
train.head(10)

,air_store_id,visit_date,visitors,day,dow,year,month,day_of_week,holiday_flg,min_visitors,...,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_ba937bf13d40fb24,2016-01-13,25,13,2,2016,1,7,0,7.0,...,64.0,25.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,14,3,2016,1,5,0,2.0,...,65.0,11.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
2,air_ba937bf13d40fb24,2016-01-15,29,15,4,2016,1,1,0,4.0,...,65.0,46.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
3,air_ba937bf13d40fb24,2016-01-16,22,16,5,2016,1,3,0,6.0,...,66.0,23.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
4,air_ba937bf13d40fb24,2016-01-18,6,18,0,2016,1,2,0,2.0,...,57.0,7.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
5,air_ba937bf13d40fb24,2016-01-19,9,19,1,2016,1,6,0,5.0,...,62.0,16.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
6,air_ba937bf13d40fb24,2016-01-20,31,20,2,2016,1,7,0,7.0,...,64.0,25.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
7,air_ba937bf13d40fb24,2016-01-21,21,21,3,2016,1,5,0,2.0,...,65.0,11.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
8,air_ba937bf13d40fb24,2016-01-22,18,22,4,2016,1,1,0,4.0,...,65.0,46.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0
9,air_ba937bf13d40fb24,2016-01-23,26,23,5,2016,1,3,0,6.0,...,66.0,23.0,4.0,62.0,35.658068,139.751599,-1.0,-1.0,-1.0,-1.0


In [233]:
test.head(10)

,id,visitors,visit_date,air_store_id,day,dow,year,month,day_of_week,holiday_flg,...,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,23,6,2017,4,4,0,...,1.0,2.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,24,0,2017,4,2,0,...,35.0,18.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,25,1,2017,4,6,0,...,40.0,20.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,26,2,2017,4,7,0,...,40.0,17.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,27,3,2017,4,5,0,...,38.0,34.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
5,air_00a91d42b08b08d9_2017-04-28,0,2017-04-28,air_00a91d42b08b08d9,28,4,2017,4,1,0,...,40.0,39.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
6,air_00a91d42b08b08d9_2017-04-29,0,2017-04-29,air_00a91d42b08b08d9,29,5,2017,4,3,1,...,38.0,7.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
7,air_00a91d42b08b08d9_2017-04-30,0,2017-04-30,air_00a91d42b08b08d9,30,6,2017,4,4,0,...,1.0,2.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
8,air_00a91d42b08b08d9_2017-05-01,0,2017-05-01,air_00a91d42b08b08d9,1,0,2017,5,2,0,...,35.0,18.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0
9,air_00a91d42b08b08d9_2017-05-02,0,2017-05-02,air_00a91d42b08b08d9,2,1,2017,5,6,0,...,40.0,20.0,6,44,35.694003,139.753595,-1.0,-1.0,-1.0,-1.0


In [234]:
print('Binding to float32')

for c, dtype in zip(train.columns, train.dtypes):
    if dtype == np.float64:
        train[c] = train[c].astype(np.float32)

for c, dtype in zip(test.columns, test.dtypes):
    if dtype == np.float64:
        test[c] = test[c].astype(np.float32)

Binding to float32


In [235]:
train.head(20)

,air_store_id,visit_date,visitors,day,dow,year,month,day_of_week,holiday_flg,min_visitors,...,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_ba937bf13d40fb24,2016-01-13,25,13,2,2016,1,7,0,7.0,...,64.0,25.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
1,air_ba937bf13d40fb24,2016-01-14,32,14,3,2016,1,5,0,2.0,...,65.0,11.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
2,air_ba937bf13d40fb24,2016-01-15,29,15,4,2016,1,1,0,4.0,...,65.0,46.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
3,air_ba937bf13d40fb24,2016-01-16,22,16,5,2016,1,3,0,6.0,...,66.0,23.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
4,air_ba937bf13d40fb24,2016-01-18,6,18,0,2016,1,2,0,2.0,...,57.0,7.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
5,air_ba937bf13d40fb24,2016-01-19,9,19,1,2016,1,6,0,5.0,...,62.0,16.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
6,air_ba937bf13d40fb24,2016-01-20,31,20,2,2016,1,7,0,7.0,...,64.0,25.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
7,air_ba937bf13d40fb24,2016-01-21,21,21,3,2016,1,5,0,2.0,...,65.0,11.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
8,air_ba937bf13d40fb24,2016-01-22,18,22,4,2016,1,1,0,4.0,...,65.0,46.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
9,air_ba937bf13d40fb24,2016-01-23,26,23,5,2016,1,3,0,6.0,...,66.0,23.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0


In [236]:
test.head(20)

,id,visitors,visit_date,air_store_id,day,dow,year,month,day_of_week,holiday_flg,...,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,23,6,2017,4,4,0,...,1.0,2.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,24,0,2017,4,2,0,...,35.0,18.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,25,1,2017,4,6,0,...,40.0,20.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,26,2,2017,4,7,0,...,40.0,17.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,27,3,2017,4,5,0,...,38.0,34.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
5,air_00a91d42b08b08d9_2017-04-28,0,2017-04-28,air_00a91d42b08b08d9,28,4,2017,4,1,0,...,40.0,39.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
6,air_00a91d42b08b08d9_2017-04-29,0,2017-04-29,air_00a91d42b08b08d9,29,5,2017,4,3,1,...,38.0,7.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
7,air_00a91d42b08b08d9_2017-04-30,0,2017-04-30,air_00a91d42b08b08d9,30,6,2017,4,4,0,...,1.0,2.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
8,air_00a91d42b08b08d9_2017-05-01,0,2017-05-01,air_00a91d42b08b08d9,1,0,2017,5,2,0,...,35.0,18.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
9,air_00a91d42b08b08d9_2017-05-02,0,2017-05-02,air_00a91d42b08b08d9,2,1,2017,5,6,0,...,40.0,20.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0


In [237]:
train_x = train.drop(['air_store_id', 'visit_date', 'visitors'], axis=1)
train_y = np.log1p(train['visitors'].values)
print(train_x.shape, train_y.shape)

(252108, 20) (252108,)


In [238]:
test_x = test.drop(['id', 'air_store_id', 'visit_date', 'visitors'], axis=1)

In [239]:
train_x.head(10)

,day,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,13,2,2016,1,7,0,7.0,23.843750,25.0,57.0,64.0,25.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
1,14,3,2016,1,5,0,2.0,20.292307,21.0,54.0,65.0,11.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
2,15,4,2016,1,1,0,4.0,34.738461,35.0,61.0,65.0,46.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
3,16,5,2016,1,3,0,6.0,27.651516,27.0,53.0,66.0,23.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
4,18,0,2016,1,2,0,2.0,13.754386,12.0,34.0,57.0,7.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
5,19,1,2016,1,6,0,5.0,18.580645,19.0,35.0,62.0,16.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
6,20,2,2016,1,7,0,7.0,23.843750,25.0,57.0,64.0,25.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
7,21,3,2016,1,5,0,2.0,20.292307,21.0,54.0,65.0,11.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
8,22,4,2016,1,1,0,4.0,34.738461,35.0,61.0,65.0,46.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0
9,23,5,2016,1,3,0,6.0,27.651516,27.0,53.0,66.0,23.0,4.0,62.0,35.65807,139.751602,-1.0,-1.0,-1.0,-1.0


In [240]:
train_y

array([ 3.25809654,  3.49650756,  3.40119738, ...,  2.07944154,
        2.19722458,  1.79175947])

In [241]:
test_x.head()

,day,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,23,6,2017,4,4,0,2.0,2.000000,2.0,2.0,1.0,2.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
1,24,0,2017,4,2,0,1.0,22.457144,19.0,47.0,35.0,18.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
2,25,1,2017,4,6,0,1.0,24.350000,24.5,43.0,40.0,20.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
3,26,2,2017,4,7,0,15.0,28.125000,28.0,52.0,40.0,17.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
4,27,3,2017,4,5,0,15.0,29.868422,30.0,47.0,38.0,34.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0


In [242]:
train_x = train_x.drop(['day_of_week'], axis=1)

In [243]:
test_x = test_x.drop(['day_of_week'], axis=1)

In [244]:
test_x.head(10)

,day,dow,year,month,holiday_flg,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,mode_visitors,air_genre_name,air_area_name,latitude,longitude,reserve_datetime_diff_x,reserve_visitors_x,reserve_datetime_diff_y,reserve_visitors_y
0,23,6,2017,4,0,2.0,2.000000,2.0,2.0,1.0,2.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
1,24,0,2017,4,0,1.0,22.457144,19.0,47.0,35.0,18.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
2,25,1,2017,4,0,1.0,24.350000,24.5,43.0,40.0,20.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
3,26,2,2017,4,0,15.0,28.125000,28.0,52.0,40.0,17.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
4,27,3,2017,4,0,15.0,29.868422,30.0,47.0,38.0,34.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
5,28,4,2017,4,0,17.0,36.500000,35.5,57.0,40.0,39.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
6,29,5,2017,4,1,3.0,14.973684,11.0,99.0,38.0,7.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
7,30,6,2017,4,0,2.0,2.000000,2.0,2.0,1.0,2.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
8,1,0,2017,5,0,1.0,22.457144,19.0,47.0,35.0,18.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0
9,2,1,2017,5,0,1.0,24.350000,24.5,43.0,40.0,20.0,6,44,35.694004,139.753601,-1.0,-1.0,-1.0,-1.0


In [245]:
print(train_x.shape, train_y.shape)

(252108, 19) (252108,)


In [246]:
# parameter tuning of lightgbm
# start from default setting
gbm0 = lgb.LGBMRegressor(
    objective='regression',
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=10000)

gbm0.fit(train_x, train_y, eval_metric='rmse')
predict_y = gbm0.predict(test_x)

In [145]:
#predict_y[predict_y<0] = 0

In [247]:
test['visitors'] = np.ceil(np.expm1(predict_y))
test['visitors'].head()

0     2.0
1    17.0
2    26.0
3    29.0
4    33.0
Name: visitors, dtype: float64

In [248]:
#test['visitors'] = np.around(np.expm1(predict_y))
test[['id', 'visitors']].to_csv(
    'gbm0_submission_ceil_new4.csv', index=False, float_format='%.3f')  # LB0.493

In [249]:
print(test['visitors'].min())
print(test['visitors'].max())
print(test['visitors'].mean())

1.0
209.0
18.5207533027
